# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [65]:
import os
import io
import numpy as np
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

train_data = DataFrame({'message': [], 'class': []}) # dataframe has two columns 'message', 'class'
test_data = DataFrame({'message': [], 'class': []})

train_data = train_data.append(dataFrameFromDirectory('./emails/spam', 'spam'))
train_data = train_data.append(dataFrameFromDirectory('./emails/ham', 'ham'))
test_data = test_data.append(dataFrameFromDirectory('./emails/test_spam', 'spam'))
print(len(test_data['message']))
test_data = test_data.append(dataFrameFromDirectory('./emails/test_ham', 'ham'))
print(len(test_data['message']))


122
347


Let's have a look at that DataFrame:

In [62]:
data.head()

,class,message
./emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a,spam,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
./emails/spam/00373.ebe8670ac56b04125c25100a36ab0510,spam,ATTENTION: This is a MUST for ALL Computer Use...
./emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216,spam,This is a multi-part message in MIME format.\n...
./emails/spam/00210.050ffd105bd4e006771ee63cabc59978,spam,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
./emails/spam/00033.9babb58d9298daa2963d4f514193d7d6,spam,This is the bottom line. If you can GIVE AWAY...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [63]:
vectorizer = CountVectorizer()
# convert data to number and count how many times certain word occured
counts = vectorizer.fit_transform(train_data['message'].values) 

classifier = MultinomialNB()
targets = train_data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [75]:
def howMuchSpamPercent(categorized):
    total = len(categorized)
    spam = 0
    for result in categorized:
        if result == 'spam':
            spam += 1
    return spam / total

examples = [
    'Free Viagra now!!!',
    "Hi Bob, how about a game of golf tomorrow?",
    "hello, you are chosen as a lucky guy!! buy this product now",
    "You can get it at:",
    "buy now or you will die",
    "Rx Shopping Service Brings You our Newest Product:",
    "100% FREE"
]

spam_emails = test_data['message'][:122]
ham_emails = test_data['message'][122:]

spam_example_counts = vectorizer.transform(spam_emails)
ham_example_counts = vectorizer.transform(ham_emails)
out_of_spam_predictions = classifier.predict(spam_example_counts)
out_of_ham_predictions = classifier.predict(ham_example_counts)
out_of_spam_predictions
print(howMuchSpamPercent(out_of_spam_predictions), howMuchSpamPercent(out_of_ham_predictions))

0.6885245901639344 0.0


## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.